In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [26]:
url = 'https://c.cari.com.my/forum.php'

In [27]:
r = requests.get(url)

In [28]:
soup = BeautifulSoup(r.content, "lxml")

In [31]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if 'forum.php?mod=forumdisplay&fid=' in a_]
forums = list(set(a))

In [32]:
with open('forums.json', 'w') as fopen:
    json.dump(forums, fopen)

In [41]:
url = 'https://c.cari.com.my/' + forums[0]
url

'https://c.cari.com.my/forum.php?mod=forumdisplay&fid=17'

In [44]:
forums[0]

'forum.php?mod=forumdisplay&fid=17'

In [45]:
forum = forums[0]

In [49]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if forum in a_ and '&page=' in a_]
try:
    max_page = max([int(a_.split('page=')[-1]) for a_ in a])
except:
    max_page = -1
max_page

1882

In [54]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_.split('&extra=')[0] for a_ in a if a_.startswith('forum.php?mod=viewthread&tid=')]
a = list(set(a))
a

['forum.php?mod=viewthread&tid=5067459',
 'forum.php?mod=viewthread&tid=5067441',
 'forum.php?mod=viewthread&tid=5065528',
 'forum.php?mod=viewthread&tid=1452056',
 'forum.php?mod=viewthread&tid=5067695',
 'forum.php?mod=viewthread&tid=5059256',
 'forum.php?mod=viewthread&tid=5067351',
 'forum.php?mod=viewthread&tid=5066529',
 'forum.php?mod=viewthread&tid=5064572',
 'forum.php?mod=viewthread&tid=5064930',
 'forum.php?mod=viewthread&tid=5067458',
 'forum.php?mod=viewthread&tid=5061878',
 'forum.php?mod=viewthread&tid=5066061',
 'forum.php?mod=viewthread&tid=5067820',
 'forum.php?mod=viewthread&tid=5067313',
 'forum.php?mod=viewthread&tid=5064722',
 'forum.php?mod=viewthread&tid=5067696',
 'forum.php?mod=viewthread&tid=5067396',
 'forum.php?mod=viewthread&tid=5057722',
 'forum.php?mod=viewthread&tid=5067658',
 'forum.php?mod=viewthread&tid=5067903',
 'forum.php?mod=viewthread&tid=5067700',
 'forum.php?mod=viewthread&tid=5066859',
 'forum.php?mod=viewthread&tid=5067703',
 'forum.php?mod=

In [55]:
def get_href(url, forum):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if forum in a_ and '&page=' in a_]
    try:
        max_page = max([int(a_.split('page=')[-1]) for a_ in a])
    except:
        max_page = -1
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_.split('&extra=')[0] for a_ in a if a_.startswith('forum.php?mod=viewthread&tid=')]
    a = list(set(a))
        
    return a, max_page

filtered, max_page = get_href(url, forums[0])
filtered, max_page

(['forum.php?mod=viewthread&tid=5067459',
  'forum.php?mod=viewthread&tid=5067441',
  'forum.php?mod=viewthread&tid=5065528',
  'forum.php?mod=viewthread&tid=1452056',
  'forum.php?mod=viewthread&tid=5067695',
  'forum.php?mod=viewthread&tid=5059256',
  'forum.php?mod=viewthread&tid=5067351',
  'forum.php?mod=viewthread&tid=5066529',
  'forum.php?mod=viewthread&tid=5064572',
  'forum.php?mod=viewthread&tid=5064930',
  'forum.php?mod=viewthread&tid=5067458',
  'forum.php?mod=viewthread&tid=5061878',
  'forum.php?mod=viewthread&tid=5066061',
  'forum.php?mod=viewthread&tid=5067820',
  'forum.php?mod=viewthread&tid=5067313',
  'forum.php?mod=viewthread&tid=5064722',
  'forum.php?mod=viewthread&tid=5067696',
  'forum.php?mod=viewthread&tid=5067396',
  'forum.php?mod=viewthread&tid=5057722',
  'forum.php?mod=viewthread&tid=5067658',
  'forum.php?mod=viewthread&tid=5067903',
  'forum.php?mod=viewthread&tid=5067700',
  'forum.php?mod=viewthread&tid=5066859',
  'forum.php?mod=viewthread&tid=50

In [57]:
!mkdir forums

In [64]:
max_worker = 30

forums = sorted(forums)
for i in tqdm(range(len(forums))):
    filename = os.path.join('forums', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    forum = forums[i]
    url = 'https://c.cari.com.my/' + forum
    filtered, max_page = get_href(url, forum)
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{url}&page={a}', forum) for a in aranged]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href, url[0], url[1]): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

  0%|                                                                                                         | 0/14 [00:00<?, ?it/s]


  7%|██████▍                                                                                          | 6/90 [03:05<52:11, 37.28s/it]
0it [00:00, ?it/s]
  8%|███████▌                                                                                         | 7/90 [03:06<35:05, 25.37s/it]
0it [00:00, ?it/s]
  9%|████████▌                                                                                        | 8/90 [03:06<23:51, 17.45s/it]
0it [00:00, ?it/s]
  9%|████████▋                                                                                        | 6/67 [00:16<03:12,  3.16s/it]


 97%|█████████████████████████████████████████████████████████████████████████████████████████████▏  | 65/67 [03:46<00:08,  4.13s/it]


 49%|███████████████████████████████████████████████                                                 | 26/53 [01:07<01:13,  2.74s/it]


 36%|███████████████████████████████████▎                                                             | 4/11 [00:07<00:12,  1.77s/it]


...

 61%|██████████████████████████▎                | 41/67 [01:23<01:01,  2.35s/it]


 19%|████████▎                                   | 6/32 [00:15<01:05,  2.52s/it]


 30%|████████████▊                              | 14/47 [00:26<01:03,  1.92s/it]


 26%|███████████▌                                | 5/19 [00:14<00:40,  2.88s/it]


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [1:11:16<00:00, 47.51s/it]
